In [1]:
# 忽略各模块的警告信息
import warnings
warnings.filterwarnings("ignore")

In [2]:
# 加载所需使用的模块
from datetime import datetime

from vnpy.trader.datafeed import get_datafeed
from vnpy.trader.database import get_database, DB_TZ
from vnpy.trader.constant import Interval
from vnpy.trader.object import BarData, HistoryRequest
from vnpy.trader.utility import extract_vt_symbol
from vnpy.trader.setting import SETTINGS

In [ ]:
# 配置数据服务
SETTINGS["datafeed.name"] = "rqdata"            # 可以根据自己的需求选择数据服务：rqdata/xt/wind等
SETTINGS["datafeed.username"] = "license"       # RQData的用户名统一为“license”这个字符串
SETTINGS["datafeed.password"] = "123456"        # 这里需要替换为你购买或者申请试用的RQData数据license

# 配置数据库
SETTINGS["database.name"] = "taos"              # 可以根据自己的需求选择数据库，这里使用的是TDengine
SETTINGS["database.database"] = "vnpy"
SETTINGS["database.host"] = "127.0.0.1"
SETTINGS["database.port"] = 6030
SETTINGS["database.user"] = "root"
SETTINGS["database.password"] = "taosdata"

In [3]:
# 创建对象实例
datafeed = get_datafeed()

database = get_database()

In [4]:
# 要下载数据的合约代码
vt_symbols = [
    "IF2501.CFFEX",
    "IF2502.CFFEX",
    "IF2503.CFFEX",

    "IH2501.CFFEX",
    "IH2502.CFFEX",
    "IH2503.CFFEX",

    "IC2501.CFFEX",
    "IC2502.CFFEX",
    "IC2503.CFFEX",

    "IM2501.CFFEX",
    "IM2502.CFFEX",
    "IM2503.CFFEX",
]

In [5]:
# 要下载数据的起止时间
start = datetime(2025, 1, 1, tzinfo=DB_TZ)
end = datetime(2025, 3, 30, tzinfo=DB_TZ)

In [6]:
# 遍历列表执行下载
for vt_symbol in vt_symbols:
    # 拆分合约代码和交易所
    symbol, exchange = extract_vt_symbol(vt_symbol)

    # 创建历史数据请求对象
    req: HistoryRequest = HistoryRequest(
        symbol=symbol,
        exchange=exchange,
        start=start,
        end=end,
        interval=Interval.MINUTE        # 这里下载最常用的1分钟K线
    )

    # 从数据服务下载数据
    bars: list[BarData] = datafeed.query_bar_history(req)

    # 如果下载成功则保存
    if bars:
        database.save_bar_data(bars)
        print(f"下载数据成功：{vt_symbol}，总数据量：{len(bars)}")
    # 否则失败则打印信息
    else:
        print(f"下载数据失败：{vt_symbol}")

下载数据成功：IF2501.CFFEX，总数据量：2880
下载数据成功：IF2502.CFFEX，总数据量：7440
下载数据成功：IF2503.CFFEX，总数据量：12240
下载数据成功：IH2501.CFFEX，总数据量：2880
下载数据成功：IH2502.CFFEX，总数据量：7440
下载数据成功：IH2503.CFFEX，总数据量：12240
下载数据成功：IC2501.CFFEX，总数据量：2880
下载数据成功：IC2502.CFFEX，总数据量：7440
下载数据成功：IC2503.CFFEX，总数据量：12240
下载数据成功：IM2501.CFFEX，总数据量：2880
下载数据成功：IM2502.CFFEX，总数据量：7440
下载数据成功：IM2503.CFFEX，总数据量：12240
